#Profiling Pecan Consumers

This notebook profiles the consumers in the Pecan St dataset who have HVAC using a thermal regimes model using whole-home consumption and temperature

##Initializations

In [37]:
rm(list = ls())
options(error = recover)
library('segmented')
library('lubridate')
library('zoo')

In [38]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')

setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")

In [39]:
DATA_PATH = '~/S3L_server/energy-data/pecan_street/usage-select/'
DUMP_PATH = '~/S3L_server/energy-data/pecan_street/models_2/'
PLOT_PATH = '~/S3L_server/plots/pecan-street-2/'

In [40]:
# load user names
user_names = read.csv('~/S3L_server/energy-data/pecan_street/metadata/user_names_ids.csv')
user_names$X = NULL

# list already processed files
files.input = list.files(path=DUMP_PATH, pattern = '*_decoded*', full.names = T, recursive = T)
already_done  = lapply(files.input, function(x) {
    tmp = tail(strsplit(x, '/')[[1]],n=2)
    res = tmp[1]
    nfo = strsplit(tmp[2], "_")[[1]]
    uid = nfo[1]; nfo = nfo[2]; 
    return(c(uid,res))
})
already_done = data.frame(do.call('rbind', already_done))
if (length(already_done)>0) names(already_done) <- c("ID", "grain")

# list all data files
files    = list.files(path=DATA_PATH, full.names = T, recursive = T)
# files_01 = files[grep('01min',files)]
# files_15 = files[grep('15min',files)]
files_60 = files[grep('60min',files)]

# extract ID
users_df = data.frame(UID = as.character(sapply(files_60, function(s) strsplit(tail(strsplit(s, '/')[[1]], 1), '\\.')[[1]][1])))
rownames(users_df) = NULL
# users_df['file_01min'] = files_01
# users_df['15min'] = files_15
users_df['60min'] = files_60
    
# build data sources dataframe
users_df = merge(user_names, users_df, by.x="ID", by.y="UID")
users_df = melt(users_df, id=c("ID","name"))
names(users_df)[c(3,4)] = c("grain", "file_orig")  
    
# filter out those IDS already done
users_df = subset(users_df, !((ID %in% already_done$ID & grain %in% already_done$grain)))
print(paste("To process:", dim(users_df)[1], "files"))

[1] "To process: 123 files"


In [41]:
head(users_df)

,ID,name,grain,file_orig
1,22,Christian,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/22.csv
49,1479,Abe,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/1479.csv
62,1782,David,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/1782.csv
64,1801,Richard,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/1801.csv
80,2242,Alfred,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/2242.csv
82,2335,Sam,60min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/2335.csv


##Learn thermal regimes models

In [42]:
# function to format data to work with the R thermal profiler code
format_data = function(homeData) {

    # interpolate small gaps
    cat(paste("Initially nrows=",nrow(homeData), '\n'))
    homeData$total = na.spline(homeData$total, maxgap = 12, na.rm=FALSE)
    
    # remove observations with NAs in response
    idx.na = is.na(homeData$total)
    if (sum(idx.na)>0) homeData = homeData[!idx.na,]
    cat(paste("\tRemoved NAs: nrows=",nrow(homeData),"\n"))

    # temperature above reference
    homeData$TemperatureD = homeData$Temperature - 65

    # format data as expected by the HMM package
    cur_data = subset(homeData, select = c('date', 'total'))
    names(cur_data)[2] = 'obs'
    cur_data$date = as.character(cur_data$date)
    cur_covar = subset(homeData, select = c('date', 'Temperature', 'TemperatureD'))
    cur_covar$date = as.character(cur_covar$date)
    cur_month     = month(cur_data$date)
    cur_covar$TemperatureDWinter = cur_covar$TemperatureD * (cur_month %in% c(0,1,2,3,10,11,12))

    return(list(cur_data, cur_covar))
}


In [43]:
apply_thermal_model = function(cur_data, cur_covar, userName, 
                               dump_path = NULL, 
                               plot_path = NULL) {
  
    # define model learning controls
    controls = list(
        Kmin = 2, Kmax = 6, 
        maxit = 50, 
        nRestarts = 5, 
        tol = 1e-5,
        thresh.R2 = 0.75, 
        thresh.MAPE = 0.25,
        test.periods = 12,
        vis.interval = 3 * 24
    )

    # generate visualization interval; make sure there's data in there
    # TODO: there was an error generated here (indices for subsetting were messed up)
    ok = FALSE
    no.secs    = controls$vis.interval * 3600
    while (!ok) {
        idx_start  = 1
        idx_end    = max(nrow(cur_data)-controls$vis.interval-1, 1)
        start_date = sample(cur_data$date[idx_start:idx_end], 1)
        stop_date  = as.character(as.POSIXct(start_date) + no.secs)
        dat        = subset(cur_data, date >= start_date & date < stop_date)
        if (nrow(na.omit(dat)) > 0) 
          ok = TRUE
    }        

    # learn model
    res = try(stateProcessorWrapper(cur_data, 
                                    cur_covar, 
                                    userName, 
                                    controls = controls,
                                    train.frac = 1.0, 
                                    verbose = F, 
                                    tran.vars  = c('(Intercept)', 'Temperature'),
                                    resp.vars = c('(Intercept)', 
                                                  'TemperatureD'), 
                                                  #'TemperatureDWinter'),
                                    dump_path = dump_path))
    if (class(res) == 'try-error') {
        cat(paste('Error in learning model for', userName,'! \n'))
        return(NULL)
    }

    # produce visualizations
    if (!is.null(plot_path)) {
        resv = stateVisualizerWrapper(res$decoder, 
                                   res$interpreter, 
                                   plots_path = plot_path, 
                                   interval = c(start_date, stop_date))
        if (class(resv) == 'try-error') {
            cat('Error in visualizing current user!\n')
        }
    }
    return(NULL)
}


In [44]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')
    
setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")

In [45]:
which(users_df$name=='Lucius' & users_df$grain=='60min')

integer(0)

In [49]:
nrow(users_df)

[1] 123

In [47]:
users_df = users_df[with(users_df, order(ID,grain,name)),]
for (i in 1:nrow(users_df)) {
    cat(paste("*****", i, ":", users_df[i,c("name", "ID")], "*****"))
    
    # load data   
    user_id = paste(users_df[i,"ID" ], users_df[i,"name"], sep="_")
    cat(paste('Processing file', users_df[i,'file_orig'], ':', i, '/', nrow(users_df), '\n'))  
    homeData = read.csv(users_df[i,"file_orig"])     
    
    # is there enough data?
    if (is.null(homeData))  {
        cat('Too little data!\n')
        return(NULL)
    }  
    
    # create directory to store models
    dump_path = file.path(DUMP_PATH, paste(user_id, users_df[i,"grain"], sep='/')); 
    dir.create(dump_path, recursive = T)
    # create directory to store plots
    plot_path = file.path(PLOT_PATH, paste(user_id, users_df[i,"name"], sep='/')); 
    dir.create(plot_path, recursive = T)

    # format dataset
    res = format_data(homeData); cur_data = res[[1]]; cur_covar = res[[2]];
    
    # apply model to data
    res = apply_thermal_model(cur_data, cur_covar, user_id, 
                            dump_path = dump_path, 
                            plot_path = plot_path)  
}

***** 1 : 100 ***** ***** 1 : 22 *****Processing file /Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/22.csv : 1 / 123 


Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_2//22_Christian/60min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-2//22_Christian/Christian' already exists

Initially nrows= 6833 
	Removed NAs: nrows= 6833 
*** Initializing DataFormatter (22_Christian) ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Adding covariates data for DataFormatter22_Christian ***
   user  system elapsed 
  0.003   0.000   0.003 
*** Initializing StateDecoder (22_Christian) ***
   user  system elapsed 
  0.002   0.000   0.002 
***** HMM Cross-Validation K = 2 ******
Bad HMM fit; re-estimating...
Bad HMM fit; re-estimating...
iteration 0 logLik: -3444.003 
iteration 5 logLik: -2107.742 
converged at iteration 9 with logLik: -2103.12 
[1] "MAPE=0.639370099889984; R2=0.136701754522157"
***** HMM Cross-Validation K = 3 ******
iteration 0 logLik: -1957.74 
iteration 5 logLik: -1149.015 
iteration 10 logLik: -983.0926 
iteration 15 logLik: -970.4184 
iteration 20 logLik: -969.6003 
converged at iteration 23 with logLik: -969.548 
[1] "MAPE=0.332889566130642; R2=-1.27332474949046"
***** HMM Cross-Validation K = 4 ******
iteration 0 logLik: -1960.333 
iteration 5

Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_2//1479_Abe/60min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-2//1479_Abe/Abe' already exists

Initially nrows= 3240 
	Removed NAs: nrows= 3240 
*** Initializing DataFormatter (1479_Abe) ***
   user  system elapsed 
      0       0       0 
*** Adding covariates data for DataFormatter1479_Abe ***
   user  system elapsed 
  0.000   0.000   0.001 
*** Initializing StateDecoder (1479_Abe) ***
   user  system elapsed 
  0.001   0.000   0.001 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -483.4826 
iteration 5 logLik: 413.9357 
Bad HMM fit; re-estimating...
iteration 0 logLik: -486.7376 
iteration 5 logLik: 421.4243 
Bad HMM fit; re-estimating...
iteration 0 logLik: -500.1014 
iteration 5 logLik: 391.4328 
Bad HMM fit; re-estimating...
iteration 0 logLik: -519.9379 
iteration 5 logLik: 318.6501 
Bad HMM fit; re-estimating...
iteration 0 logLik: -516.8822 
iteration 5 logLik: 345.7362 
Bad HMM fit; re-estimating...
iteration 0 logLik: -529.8615 
iteration 5 logLik: 66.84378 
iteration 10 logLik: 616.8701 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cros

...

***** 92 : 108 ***** ***** 92 : 8168 *****Processing file /Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//60min/8168.csv : 92 / 123 
Initially nrows= 2519 
	Removed NAs: nrows= 2519 
*** Initializing DataFormatter (8168_Clay) ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Adding covariates data for DataFormatter8168_Clay ***
   user  system elapsed 
      0       0       0 
*** Initializing StateDecoder (8168_Clay) ***
   user  system elapsed 
  0.001   0.000   0.001 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -340.2861 
iteration 5 logLik: -15.24073 
iteration 10 logLik: 149.2327 
iteration 15 logLik: 166.0476 
iteration 20 logLik: 168.2776 
iteration 25 logLik: 168.6359 
iteration 30 logLik: 168.6905 
converged at iteration 33 with logLik: 168.6968 
[1] "MAPE=0.239666208572113; R2=0.527816564755177"
iteration 0 logLik: -628.0807 
iteration 5 logLik: 471.6838 
iteration 10 logLik: 643.7968 
Bad HMM fit; re-estimating...
iteration 0 log

Warning message:
In matrix(as.numeric(myMat), ncol = 24, byrow = T): data length [2519] is not a sub-multiple or multiple of the number of rows [105]

      obs           State        Index                 Time   
 Min.   :0.2082   N (1):20   Min.   : 1   Fri,11/22 00:00: 1  
 1st Qu.:0.3636   N (2):53   1st Qu.:19   Fri,11/22 01:00: 1  
 Median :0.5549              Median :37   Fri,11/22 02:00: 1  
 Mean   :0.5976              Mean   :37   Fri,11/22 03:00: 1  
 3rd Qu.:0.8286              3rd Qu.:55   Fri,11/22 04:00: 1  
 Max.   :1.1844              Max.   :73   Fri,11/22 05:00: 1  
                                          (Other)        :67  
  TemperatureD       
 Min.   :-0.0017731  
 1st Qu.:-0.0017731  
 Median :-0.0006325  
 Mean   :-0.0009450  
 3rd Qu.:-0.0006325  
 Max.   :-0.0006325  
                     


Warning message:
: The shape palette can deal with a maximum of 6 discrete values because
more than 6 becomes difficult to discriminate; you have 7. Consider
specifying shapes manually. if you must have them.Warning message:
: The shape palette can deal with a maximum of 6 discrete values because
more than 6 becomes difficult to discriminate; you have 7. Consider
specifying shapes manually. if you must have them.Warning message:
In loop_apply(n, do.ply): Removed 24 rows containing missing values (geom_point).Warning message:
In loop_apply(n, do.ply): Removed 24 rows containing missing values (geom_point).

ERROR: Error in mapply(FUN = f, ..., SIMPLIFY = FALSE): unimplemented type (29) in 'duplicate'



In [48]:
i

[1] 92